# Data Mining / Prospecção de Dados

## Sara C. Madeira, 2024/2025

# Project 2 - Classification in Temporal Data using Sequential Pattern Mining

## Logistics 
**_Read Carefully_**

**Students should work in teams of 3 people**. 

Groups with less than 3 people might be allowed (with valid justification), but will not have better grades for this reason. 

The quality of the project will dictate its grade, not the number of people working.

**The project's solution should be uploaded in Moodle before the end of `June, 8th (23:59)`.** 

Students should **upload a `.zip` file** containing a folder with all the files necessary for project evaluation. 
Groups should be registered in [Moodle](https://moodle.ciencias.ulisboa.pt/mod/groupselect/view.php?id=139096) and the `zip` file should be identified as `PDnn.zip` where `nn` is the number of your group.

**It is mandatory to produce a Jupyter notebook containing code and text/images/tables/etc describing the solution and the results. Projects not delivered in this format will not be graded. You can use `PD_202425_P2.ipynb` as template. In your `.zip` folder you should also include an HTML version of your notebook with all the outputs.**

**Decisions should be justified and results should be critically discussed.** 

Remember that **your notebook should be as clear and organized as possible**, that is, **only the relevant code and experiments should be presented, not everything you tried and did not work, or is not relevant** (that can be discussed in the text, if relevant)! Tables and figures can be used together with text to summarize results and conclusions, improving understanding, readability and concision. **More does not mean better! The target is quality not quantity!**

_**Project solutions containing only code and outputs without discussions will achieve a maximum grade of 10 out of 20.**_

## Dataset and Tools

Amyotrophic Lateral Sclerosis (ALS) is a devastating neurodegenerative disease causing rapid degeneration of motor neurons and usually leading to death by respiratory failure. Since there is no cure, treatment’s goal is to improve symptoms and prolong survival. Non-invasive Ventilation (NIV) has been shown to extend life expectancy and improve quality of life, thus it is key to effectively predict if ALS patients will be eligible for NIV in the near future based on disease progression. In this context, Martins et al. (2021) proposed to learn prognostic models using disesase progression patterns (https://ieeexplore.ieee.org/document/9426397), and formulated the following prognostic prediction problem (schematized in Figure below): given a specific ALS patient static data collected at diagnosis and temporal data from disease follow-up, can we effectively use these clinical evaluations to predict if this patient will require NIV within k days of last evaluation?

<img src="prognostic_problem.png" alt="Prognostic Prediction" style="width: 500px;"/>

In this project, we will perform a reduced part of the work published by Martins et al. to learn a machine learning model (classifier) able to predict the need for NIV in a time window of 180 days given static and temporal data and sequential patterns as features.

The dataset to be analysed was obtained from the Lisbon ALS database, containing clinical data from ALS patients collected during their follow-up at the hospital. To reduce the preprocessing steps relatively to what was done in the paper and should be done in the real scenario the following **preprocessed datasets** are already provided:

1. `Dataset_Static_Features.csv` - each row is a patient with a REF id described by a set of features collected at diagnosis time. These features are called static since their are not collected over time. Duplicated patients should be deleted.

<img src="static_example.png" alt="Temporal Data" style="width: 1000px;"/>

2. `Dataset_Temporal_Features.csv` - each patient has a set of rows (snapshots), each corresponding to a visit at the hospital and the values collected for a set of temporal features collected over time. The rows per patient REF are sorted in chonologic order, such that the first row and the last row of a REF correspond, respectively to the data collected in the first and last visit at the hospital. In the example below, the patient REF=9 has temporal data describing 5 clinical evaluations (time-points).

<img src="temporal_example.png" alt="Temporal Data" style="width: 500px;"/>

3. `Dataset_NIV_Evolution_180.cvs`- each patient has a set of rows with the true value of NIV (yes or no) 180 days in the future. These are the class labels to be later used to train the classifier. Patient REF=9 does not evolve to need NIV 180 after time-points 1 to 4 but evolves to NIV 180 after time-point 5.

<img src="NIV_evolution_example.png" alt="Temporal Data" style="width: 100px;"/>

In this context, the project has **2 main tasks**:
1. Learn disease progression patterns from the temporal data using temporal pattern mining
2. Learn a classifier to predict NIV using temporal patterns as features together with the static features


**In this project you should use [Python 3](https://www.python.org), [Jupyter Notebook](http://jupyter.org), [`SPMF`](https://www.philippe-fournier-viger.com/software.php) for temporal pattern mining, and [`Scikit-learn`](https://scikit-learn.org/stable/) for classification.**


## Team Identification

**GROUP 07**

Students:

* Daniel Carvalho - 64350
* Gabriel Meirinho - 64873
* Rita Silva - 56798

## 1. Learn disease progression patterns from the temporal data using temporal pattern mining

In this first task you should load and preprocessed the dataset **`Dataset_Temporal_Features.csv`** in order to compute sequential patterns for each patient. 

For that, you first need to load and preprocess this dataset and then transform the temporal data into a **sequence database**.

You should consider **a minimum of 2 time-points** and a **maximum of 5 time-points** per patient.

The sequential pattern mining algorithm `Fourier08` (https://www.philippe-fournier-viger.com/spmf/ClosedSequentialPatterns_TimeConstraints.php), an extension of `PrefixSpan` able to **deal with time** and compute **close patterns**, should be used.

In [ ]:
def alsfrs_intervals(val):
    if val < 4:
        return '<4'
    elif 4 <= val < 8:
        return '[4,8['
    elif 8 <= val < 12:
        return '[8,12['
    else:
        return '12'
    
def alsfrs_r(val):
    if val < 12:
        return '<12'
    elif 12 <= val < 24:
        return '[12,24['
    elif 24 <= val < 36:
        return '[24,36['
    else:
        return '>=36'

### 1.1. Load and Preprocess Dataset

- Remove all patients (REFs) with missing values in the temporal features.
- Remove all patients with less than 2 time-points.
- For the patients with more than 5 time-points, keep only the first 5.
- In order to reduce the number of patterns to be generated agregate the values of the temporal features using the following intervals (4 intervals for each feature as in Martins et al): 
  1) Intervals for `ALSFRSb, ALSFRSsUL, ALSFRSsT, ALSFRSsLL, and R: <4 [4,8[ [8,12[ 12`;
  2) Intervals for `ALSFRS-R: <12  [12,24[  [24,36[  >=36`

In [191]:
import pandas as pd
import os
from spmf import Spmf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
import re

In [192]:
temporal_df = pd.read_csv('Dataset_Temporal_Features.csv', delimiter=';')

In [193]:
temporal_df.head(3)

,REF,ALSFRSb,ALSFRSsUL,ALSFRSsT,ALSFRSsLL,R,ALSFRS-R
0,2,12.0,8.0,6.0,4.0,12.0,42.0
1,2,12.0,8.0,6.0,4.0,12.0,42.0
2,2,12.0,8.0,6.0,3.0,12.0,41.0


In [194]:
print(f"Number of nulls per column\n{temporal_df.isnull().sum()}")

Number of nulls per column
REF            0
ALSFRSb      318
ALSFRSsUL    329
ALSFRSsT     329
ALSFRSsLL    329
R            366
ALSFRS-R     381
dtype: int64


### Remove all patients with missing values in the temporal features

In [195]:
temporal_df = temporal_df.dropna()
print(f"\nNumber of nulls per column\n{temporal_df.isnull().sum()}")


Number of nulls per column
REF          0
ALSFRSb      0
ALSFRSsUL    0
ALSFRSsT     0
ALSFRSsLL    0
R            0
ALSFRS-R     0
dtype: int64


In [196]:
ref_counts = temporal_df['REF'].value_counts()
print(f"\nReference counts:\n{ref_counts}")


Reference counts:
REF
723     26
269     25
608     24
395     24
981     23
        ..
1162     1
1453     1
110      1
519      1
521      1
Name: count, Length: 957, dtype: int64


### Remove all patients with less than 2 time-points

In [197]:
temporal_df = temporal_df.groupby('REF').filter(lambda x: len(x) >= 2)

### For the patients with more than 5 time-points, keep only the first 5

In [ ]:
temporal_df = temporal_df.groupby('REF').head(5)
print(f"\nNumber of visits per REF after filtering:\n{temporal_df['REF'].value_counts()}")


Number of visits per REF after filtering:
REF
571     5
920     5
922     5
924     5
1539    5
       ..
1027    2
1034    2
1039    2
1043    2
1878    2
Name: count, Length: 691, dtype: int64


In [199]:
#print the unique values in all columns except 'REF'
for col in temporal_df.columns:
    if col != 'REF':
        unique_values = temporal_df[col].unique()
        print(f"\nUnique values in column '{col}':\n{unique_values}")


Unique values in column 'ALSFRSb':
[12. 10.  9. 11.  7.  8.  6.  4.  5.  2.  0.  1.  3.]

Unique values in column 'ALSFRSsUL':
[8. 5. 6. 3. 4. 2. 7. 1. 0.]

Unique values in column 'ALSFRSsT':
[6. 4. 2. 8. 7. 3. 5. 1. 0.]

Unique values in column 'ALSFRSsLL':
[4. 3. 8. 5. 1. 0. 6. 7. 2.]

Unique values in column 'R':
[12. 11. 10.  9.  8.]

Unique values in column 'ALSFRS-R':
[42. 41. 36. 37. 38. 32. 46. 45. 44. 31. 27. 39. 35. 30. 33. 47. 43. 40.
 29. 28. 34. 25. 23. 26. 19. 24. 48. 16. 22. 21. 20. 18. 15. 17. 13. 14.
 11. 12.]


### Intervals for the features 'ALSFRSb', 'ALSFRSsUL', 'ALSFRSsT', 'ALSFRSsLL' and 'R'

In [201]:
cols = ['ALSFRSb', 'ALSFRSsUL', 'ALSFRSsT', 'ALSFRSsLL', 'R']
for col in cols:
    temporal_df[col] = temporal_df[col].apply(alsfrs_intervals)

In [202]:
temporal_df.head(3)

,REF,ALSFRSb,ALSFRSsUL,ALSFRSsT,ALSFRSsLL,R,ALSFRS-R
0,2,12,"[8,12[","[4,8[","[4,8[",12,42.0
1,2,12,"[8,12[","[4,8[","[4,8[",12,42.0
2,2,12,"[8,12[","[4,8[",<4,12,41.0


### Intervals for feature 'ALSFRS-R'

In [204]:
temporal_df['ALSFRS-R'] = temporal_df['ALSFRS-R'].apply(alsfrs_r)

In [205]:
temporal_df.head(3)

,REF,ALSFRSb,ALSFRSsUL,ALSFRSsT,ALSFRSsLL,R,ALSFRS-R
0,2,12,"[8,12[","[4,8[","[4,8[",12,>=36
1,2,12,"[8,12[","[4,8[","[4,8[",12,>=36
2,2,12,"[8,12[","[4,8[",<4,12,>=36


In [206]:
#print the unique values in all columns except 'REF'
for col in temporal_df.columns:
    if col != 'REF':
        unique_values = temporal_df[col].unique()
        print(f"\nUnique values in column '{col}':\n{unique_values}")


Unique values in column 'ALSFRSb':
['12' '[8,12[' '[4,8[' '<4']

Unique values in column 'ALSFRSsUL':
['[8,12[' '[4,8[' '<4']

Unique values in column 'ALSFRSsT':
['[4,8[' '<4' '[8,12[']

Unique values in column 'ALSFRSsLL':
['[4,8[' '<4' '[8,12[']

Unique values in column 'R':
['12' '[8,12[']

Unique values in column 'ALSFRS-R':
['>=36' '[24,36[' '[12,24[' '<12']


### 1.2. Compute the Sequence Database

Note that items have now the form `Feature=value` and you should have a sequence database with as many sequences as patients. 

Each sequence encodes the several time-points (maximum 5) of each patient.

See `Fourier08` example (https://www.philippe-fournier-viger.com/spmf/ClosedSequentialPatterns_TimeConstraints.php) to undertand the format received by the algorithm, specially the time information. 

Remember also the end of the example with PrefixSpan (https://www.philippe-fournier-viger.com/spmf/PrefixSpan.php) to understand how to use strings instead of integers to encode items .

In [207]:
#def create_sequence_db_df(df):
#    records = []
#    item_mapping = {}
#    item_counter = 1
#
#    for ref, group in df.groupby('REF'):
#        parts = []       # for SPMF encoding
#        parts_raw = []   # for human‐readable
#        for t, (_, row) in enumerate(group.iterrows()):
#            # build encoded item list
#            items = []
#            for col in df.columns:
#                if col == 'REF':
#                    continue
#                value = str(row[col])
#                key = f"{col}={value}"
#                if key not in item_mapping:
#                    item_mapping[key] = item_counter
#                    item_counter += 1
#                items.append(str(item_mapping[key]))
#            parts.append(f"<{t}> " + " ".join(items) + " -1")
#
#            # build raw tuple
#            raw_items = " ".join(f"{col} = {row[col]}"
#                                 for col in df.columns if col != 'REF')
#            parts_raw.append(f"({t}, {raw_items} )")
#
#        sequence_str = " ".join(parts) + " -2"
#        sequence_raw = " ".join(parts_raw)
#        records.append({
#            'REF': ref,
#            'Sequence': sequence_str,
#            'Sequence_raw': sequence_raw
#        })
#
#    # build mapping header
#    mapping_str = "@CONVERTED_FROM_TEXT\n"
#    for key, number in item_mapping.items():
#        mapping_str += f"@ITEM={number}={key}\n"
#
#    return pd.DataFrame(records), mapping_str

In [208]:
#sequences, map = create_sequence_db_df(temporal_df)

In [209]:
def create_sequence_db_df(df):
    records = []
    item_mapping = {}
    item_counter = 1

    # Define all possible feature-value pairs
    all_features = {
        'ALSFRSb': ['<4', '[4,8[', '[8,12[', '12'],
        'ALSFRSsUL': ['<4', '[4,8[', '[8,12[', '12'],
        'ALSFRSsT': ['<4', '[4,8[', '[8,12[', '12'],
        'ALSFRSsLL': ['<4', '[4,8[', '[8,12[', '12'],
        'R': ['<4', '[4,8[', '[8,12[', '12'],
        'ALSFRS-R': ['<12', '[12,24[', '[24,36[', '>=36']
    }

    # Pre-populate item_mapping with all possible items
    for feature, values in all_features.items():
        for value in values:
            key = f"{feature}={value}"
            if key not in item_mapping:
                item_mapping[key] = item_counter
                item_counter += 1

    for ref, group in df.groupby('REF'):
        parts = []       # for SPMF encoding
        parts_raw = []   # for human‐readable
        for t, (_, row) in enumerate(group.iterrows()):
            # build encoded item list
            items = []
            for col in df.columns:
                if col == 'REF':
                    continue
                value = str(row[col])
                key = f"{col}={value}"
                items.append(str(item_mapping[key]))
            parts.append(f"<{t}> " + " ".join(items) + " -1")

            # build raw tuple
            raw_items = " ".join(f"{col} = {row[col]}"
                                 for col in df.columns if col != 'REF')
            parts_raw.append(f"({t}, {raw_items} )")

        sequence_str = " ".join(parts) + " -2"
        sequence_raw = " ".join(parts_raw)
        records.append({
            'REF': ref,
            'Sequence': sequence_str,
            'Sequence_raw': sequence_raw
        })

    # build mapping header
    mapping_str = "@CONVERTED_FROM_TEXT\n"
    for key, number in item_mapping.items():
        mapping_str += f"@ITEM={number}={key}\n"

    return pd.DataFrame(records), mapping_str

In [210]:
sequences, map = create_sequence_db_df(temporal_df)

In [211]:
#increase the display width to see the full sequence
pd.set_option('display.max_colwidth', None)

In [212]:
#print the number of patients
print(len(sequences['REF'].unique()))

691


In [213]:
sequences.head()

,REF,Sequence,Sequence_raw
0,2,<0> 4 7 10 14 20 24 -1 <1> 4 7 10 14 20 24 -1 <2> 4 7 10 13 20 24 -1 <3> 4 6 10 13 20 24 -1 -2,"(0, ALSFRSb = 12 ALSFRSsUL = [8,12[ ALSFRSsT = [4,8[ ALSFRSsLL = [4,8[ R = 12 ALSFRS-R = >=36 ) (1, ALSFRSb = 12 ALSFRSsUL = [8,12[ ALSFRSsT = [4,8[ ALSFRSsLL = [4,8[ R = 12 ALSFRS-R = >=36 ) (2, ALSFRSb = 12 ALSFRSsUL = [8,12[ ALSFRSsT = [4,8[ ALSFRSsLL = <4 R = 12 ALSFRS-R = >=36 ) (3, ALSFRSb = 12 ALSFRSsUL = [4,8[ ALSFRSsT = [4,8[ ALSFRSsLL = <4 R = 12 ALSFRS-R = >=36 )"
1,8,<0> 4 6 10 15 20 24 -1 <1> 4 5 9 15 20 24 -1 -2,"(0, ALSFRSb = 12 ALSFRSsUL = [4,8[ ALSFRSsT = [4,8[ ALSFRSsLL = [8,12[ R = 12 ALSFRS-R = >=36 ) (1, ALSFRSb = 12 ALSFRSsUL = <4 ALSFRSsT = <4 ALSFRSsLL = [8,12[ R = 12 ALSFRS-R = >=36 )"
2,9,<0> 4 6 10 13 20 24 -1 <1> 4 5 10 14 20 24 -1 <2> 4 6 9 13 19 23 -1 <3> 4 5 10 13 20 24 -1 <4> 4 5 10 13 19 23 -1 -2,"(0, ALSFRSb = 12 ALSFRSsUL = [4,8[ ALSFRSsT = [4,8[ ALSFRSsLL = <4 R = 12 ALSFRS-R = >=36 ) (1, ALSFRSb = 12 ALSFRSsUL = <4 ALSFRSsT = [4,8[ ALSFRSsLL = [4,8[ R = 12 ALSFRS-R = >=36 ) (2, ALSFRSb = 12 ALSFRSsUL = [4,8[ ALSFRSsT = <4 ALSFRSsLL = <4 R = [8,12[ ALSFRS-R = [24,36[ ) (3, ALSFRSb = 12 ALSFRSsUL = <4 ALSFRSsT = [4,8[ ALSFRSsLL = <4 R = 12 ALSFRS-R = >=36 ) (4, ALSFRSb = 12 ALSFRSsUL = <4 ALSFRSsT = [4,8[ ALSFRSsLL = <4 R = [8,12[ ALSFRS-R = [24,36[ )"
3,10,<0> 3 7 11 15 20 24 -1 <1> 3 7 10 15 20 24 -1 <2> 3 7 11 15 19 24 -1 -2,"(0, ALSFRSb = [8,12[ ALSFRSsUL = [8,12[ ALSFRSsT = [8,12[ ALSFRSsLL = [8,12[ R = 12 ALSFRS-R = >=36 ) (1, ALSFRSb = [8,12[ ALSFRSsUL = [8,12[ ALSFRSsT = [4,8[ ALSFRSsLL = [8,12[ R = 12 ALSFRS-R = >=36 ) (2, ALSFRSb = [8,12[ ALSFRSsUL = [8,12[ ALSFRSsT = [8,12[ ALSFRSsLL = [8,12[ R = [8,12[ ALSFRS-R = >=36 )"
4,14,<0> 3 6 10 13 19 23 -1 <1> 3 5 9 13 19 23 -1 -2,"(0, ALSFRSb = [8,12[ ALSFRSsUL = [4,8[ ALSFRSsT = [4,8[ ALSFRSsLL = <4 R = [8,12[ ALSFRS-R = [24,36[ ) (1, ALSFRSb = [8,12[ ALSFRSsUL = <4 ALSFRSsT = <4 ALSFRSsLL = <4 R = [8,12[ ALSFRS-R = [24,36[ )"


In [214]:
def save_sequences_and_map(sequences_df, mapping_str, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(mapping_str)
        for seq in sequences_df['Sequence']:
            f.write(seq.strip() + '\n')

In [215]:
save_sequences_and_map(sequences, map, 'sequences.txt')

Write text in cells like this ...


### 1.3. Compute Sequential Patterns

Use `Fourier08` to compute the closed sequential patterns. Trivial patterns of length 1 should be discarded.

Note that later you need to know what are the sequences (patients) where the patterns occur (the algorithm can output that info).

In [216]:
# # min_support, min_gap of the time intervals (saltinhos entre os time points), max_gap, min_length of the sequences (quantidade de time points), max_length
os.system("java -jar spmf.jar run Fournier08-Closed+time sequences.txt output.txt 25% 0 5 0 5")

0

In [217]:
input_file = 'output.txt'
output_file = 'filtered_output.txt'

with open(input_file, 'r', encoding='utf-8') as fin, \
     open(output_file, 'w', encoding='utf-8') as fout:
    for line in fin:
        # collect all time‐point tags like <0>, <1>
        tps = re.findall(r'<\s*(\d+)\s*>', line)
        # if exactly one time‐point and it's "0", skip this line
        if len(tps) == 1 and tps[0] == '0':
            continue
        fout.write(line)

print(f"Filtered file written to {output_file}")

Filtered file written to filtered_output.txt


In [218]:
import re

def is_subsequence(pat, seq):
    i = 0
    for pset in pat:
        while i < len(seq) and not pset.issubset(seq[i]):
            i += 1
        if i == len(seq): return False
        i += 1
    return True

def parse_sequence(raw):
    out = []
    # find every "(t, …)" chunk
    for content in re.findall(r'\(\s*\d+,\s*(.*?)\s*\)', raw):
        toks = content.split()
        s = set()
        # every three tokens are feat, '=', val
        for i in range(0, len(toks), 3):
            feat, _, val = toks[i : i+3]
            s.add(f"{feat}={val}")
        out.append(s)
    return out

# then your matching block stays the same:
pattern_records = []
with open('filtered_output.txt','r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line.startswith('<'): continue

        pat_str, sup_str = line.split('#SUP:')
        enc_pattern = pat_str.strip()
        support     = int(sup_str.strip())

        # build decoded pattern
        sets = re.findall(r'<(\d+)>\s+(.*?)\s+-1', enc_pattern)
        decoded = " ".join(
            f"({t}, {' '.join(f'{f} = {v}' for f,v in (x.split('=',1),))} )"
            for t,items in sets
            for x in [" ".join(items.split())]  # no-op to line up formats
        )
        # parse the encoded sets for subsequence
        pat_sets = [ set(s.split()) for _,s in sets ]

        hits = [
            seq_row.REF
            for _, seq_row in sequences.iterrows()
            if is_subsequence(pat_sets, parse_sequence(seq_row.Sequence_raw))
        ]

        pattern_records.append({
            'pattern':         enc_pattern,
            'decoded_pattern': decoded,
            'support':         support,
            'REF':            hits
        })

df_matches = pd.DataFrame(pattern_records)

df_matches = df_matches.sort_values(
    by='support',
    ascending=False
).reset_index(drop=True)

total_sequences = len(sequences)

df_matches['support_pct'] = (df_matches['support'] / total_sequences * 100).round(2)

In [219]:
df_matches.head(3)

,pattern,decoded_pattern,support,REF,support_pct
0,<0> R=12 -1 <1> R=12 -1,"(0, R = 12 ) (1, R = 12 )",524,"[2, 8, 9, 10, 17, 18, 20, 21, 24, 30, 34, 35, 36, 39, 40, 42, 43, 45, 46, 49, 50, 54, 55, 56, 64, 66, 67, 72, 74, 78, 79, 82, 85, 88, 91, 93, 94, 100, 103, 104, 105, 111, 113, 115, 119, 122, 125, 126, 133, 136, 137, 141, 144, 145, 151, 153, 156, 161, 162, 164, 165, 166, 167, 169, 171, 173, 174, 176, 177, 178, 179, 180, 185, 196, 197, 200, 201, 202, 205, 207, 210, 211, 212, 213, 214, 215, 216, 219, 220, 227, 236, 238, 241, 242, 247, 250, 253, 256, 259, 261, ...]",75.83
1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1,"(0, ALSFRS-R = >=36 ) (1, ALSFRS-R = >=36 )",511,"[2, 8, 9, 10, 17, 21, 24, 26, 30, 34, 35, 39, 40, 42, 45, 46, 49, 50, 54, 55, 56, 60, 61, 63, 64, 66, 72, 74, 78, 79, 82, 85, 88, 91, 93, 94, 99, 100, 103, 104, 113, 115, 119, 122, 126, 134, 136, 137, 141, 151, 155, 156, 160, 161, 162, 164, 166, 167, 169, 171, 173, 174, 176, 177, 179, 185, 196, 197, 200, 202, 203, 205, 207, 208, 210, 211, 212, 213, 214, 215, 217, 220, 227, 236, 238, 241, 242, 243, 244, 247, 250, 253, 256, 259, 261, 262, 263, 265, 267, 269, ...]",73.95
2,<0> ALSFRS-R=>=36 -1 <1> R=12 -1,"(0, ALSFRS-R = >=36 ) (1, R = 12 )",502,"[2, 8, 9, 10, 17, 21, 24, 30, 34, 35, 39, 40, 42, 45, 46, 49, 50, 54, 55, 56, 64, 66, 67, 72, 74, 78, 79, 82, 85, 88, 91, 93, 94, 100, 103, 104, 105, 113, 115, 119, 122, 126, 134, 136, 137, 141, 144, 145, 151, 153, 156, 161, 162, 164, 165, 166, 167, 169, 171, 173, 174, 176, 177, 178, 179, 180, 185, 196, 197, 200, 202, 205, 207, 210, 211, 212, 213, 214, 215, 220, 227, 236, 238, 241, 242, 247, 250, 253, 256, 259, 261, 262, 263, 265, 267, 269, 271, 273, 277, 278, ...]",72.65


In [220]:
# Explode the REF column to have one row per REF-pattern combination
df_exploded = df_matches[['pattern', 'REF']].explode('REF')

# Create binary matrix using crosstab
binary_df = pd.crosstab(df_exploded['REF'], df_exploded['pattern'])

# Convert to integer type (crosstab returns counts, but since each REF-pattern pair appears once, it's already binary)
binary_df = binary_df.astype(int)

# Reset index to make REF a column instead of index
binary_df = binary_df.reset_index()

print(f"Binary matrix shape: {binary_df.shape}")
print(f"Number of patients: {len(binary_df)}")
print(f"Number of patterns: {len(binary_df.columns) - 1}")

binary_df.head()

Binary matrix shape: (691, 5163)
Number of patients: 691
Number of patterns: 5162


pattern,REF,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> R=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1",...,<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1","<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ ALSFRS-R=>=36 -1","<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ R=12 -1","<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ R=12 ALSFRS-R=>=36 -1","<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsUL=[4,8[ -1",<0> R=12 ALSFRS-R=>=36 -1 <3> R=12 -1,<0> R=12 ALSFRS-R=>=36 -1 <3> R=12 -1 <4> R=12 -1,<0> R=12 ALSFRS-R=>=36 -1 <3> R=12 ALSFRS-R=>=36 -1,<0> R=12 ALSFRS-R=>=36 -1 <4> R=12 -1
0,2,1,1,1,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,8,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
2,9,1,1,0,0,0,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
3,10,1,1,0,0,0,0,0,0,0,...,0,1,1,1,1,0,1,0,1,1
4,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Write text in cells like this ...


## 2.  Learn a classifier to predict NIV using temporal patterns as features together with the static features

In this task you should create a training set where the features are 1) the original static features `(Dataset_static_features.cvs`and 2) the sequential patterns computed above. The class labels to be used for each patient are in file `Dataset_NIV_Evolution_180.cvs`.

### 2.1. Load/Preprocess the Dataset

- Remember to delete from `Dataset_static_features.csv` the patients you deleted in step 1.1. You should only have one row per patient, thus remove repetitions.
- Remember to delete from `Dataset_NIV_Evolution_180.cvs` the patients you deleted in step 1.1.
- Note that for each patient the class label you need from `Dataset_NIV_Evolution_180.cvs` is the one corresponding to the last time-point you considered in step 1.1.

In [221]:
evolution_df = pd.read_csv('Dataset_NIV_Evolution_180.csv', delimiter=';')
static_df = pd.read_csv('Dataset_Static_Features.csv', delimiter=';')

In [222]:
#remove the rows that have a 'REF' value that is not in the binary_df
evolution_df = evolution_df[evolution_df['REF'].isin(sequences['REF'])]
static_df = static_df[static_df['REF'].isin(sequences['REF'])]

In [223]:
static_df.head(3)

,REF,Gender,Age at onset (years),Revised El Escorial Criteria,Onset,Diagnostic delay (months),BMI at 1st visit,MND familiar history,C9orf72
0,2,Male,54.92,Probable,Limbs,5.22,17.9,No,NaN
1,2,Male,54.92,Probable,Limbs,5.22,17.9,No,NaN
2,2,Male,54.92,Probable,Limbs,5.22,17.9,No,NaN


In [224]:
#indicate null values in the static_df
print(f"Number of nulls per column in static_df:\n{static_df.isnull().sum()}")

Number of nulls per column in static_df:
REF                                0
Gender                             0
Age at onset (years)               0
Revised El Escorial Criteria       0
Onset                              0
Diagnostic delay (months)          0
BMI at 1st visit                 305
MND familiar history             140
C9orf72                         1634
dtype: int64


In [225]:
static_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3853 entries, 0 to 4321
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   REF                           3853 non-null   int64  
 1   Gender                        3853 non-null   object 
 2   Age at onset (years)          3853 non-null   float64
 3   Revised El Escorial Criteria  3853 non-null   object 
 4   Onset                         3853 non-null   object 
 5   Diagnostic delay (months)     3853 non-null   float64
 6   BMI at 1st visit              3548 non-null   float64
 7   MND familiar history          3713 non-null   object 
 8   C9orf72                       2219 non-null   object 
dtypes: float64(3), int64(1), object(5)
memory usage: 301.0+ KB


In [226]:
static_df = static_df.dropna(subset=['BMI at 1st visit'])

In [227]:
static_df = static_df.fillna('unknown')
print(f"\nNumber of nulls per column in static_df after filling:\n{static_df.isnull().sum()}")


Number of nulls per column in static_df after filling:
REF                             0
Gender                          0
Age at onset (years)            0
Revised El Escorial Criteria    0
Onset                           0
Diagnostic delay (months)       0
BMI at 1st visit                0
MND familiar history            0
C9orf72                         0
dtype: int64


In [228]:
evolution_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3853 entries, 0 to 4321
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   REF        3853 non-null   int64 
 1   Evolution  3853 non-null   object
dtypes: int64(1), object(1)
memory usage: 90.3+ KB


In [229]:
#delete the C9orf72 column
#static_df = static_df.drop(columns=['C9orf72'])
#static_df = static_df.dropna()
#evaluation_df = evaluation_df.dropna()

In [230]:
len(static_df), len(evolution_df)

(3548, 3853)

In [231]:
evolution_df.head(3)

,REF,Evolution
0,2,N
1,2,N
2,2,N


In [232]:
static_df.head(3)

,REF,Gender,Age at onset (years),Revised El Escorial Criteria,Onset,Diagnostic delay (months),BMI at 1st visit,MND familiar history,C9orf72
0,2,Male,54.92,Probable,Limbs,5.22,17.9,No,unknown
1,2,Male,54.92,Probable,Limbs,5.22,17.9,No,unknown
2,2,Male,54.92,Probable,Limbs,5.22,17.9,No,unknown


In [233]:
binary_df

pattern,REF,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> R=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1",...,<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1","<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ ALSFRS-R=>=36 -1","<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ R=12 -1","<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ R=12 ALSFRS-R=>=36 -1","<0> R=12 ALSFRS-R=>=36 -1 <3> ALSFRSsUL=[4,8[ -1",<0> R=12 ALSFRS-R=>=36 -1 <3> R=12 -1,<0> R=12 ALSFRS-R=>=36 -1 <3> R=12 -1 <4> R=12 -1,<0> R=12 ALSFRS-R=>=36 -1 <3> R=12 ALSFRS-R=>=36 -1,<0> R=12 ALSFRS-R=>=36 -1 <4> R=12 -1
0,2,1,1,1,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,8,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
2,9,1,1,0,0,0,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
3,10,1,1,0,0,0,0,0,0,0,...,0,1,1,1,1,0,1,0,1,1
4,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,1795,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
687,1814,1,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,1,1
688,1818,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
689,1877,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [234]:
#display the refs that have all 0 values in the binary_df
all_zero_refs = binary_df[binary_df.drop(columns=['REF']).sum(axis=1) == 0]['REF']
print(f"References with all zero values in binary_df:\n{all_zero_refs}")

References with all zero values in binary_df:
Series([], Name: REF, dtype: int64)


In [177]:
#for each patient in the evolution_df and static_df, keep the last record for each REF
evolution_df = evolution_df.groupby('REF').tail(1)
static_df = static_df.groupby('REF').tail(1)

In [178]:
#'inner' por causa dos BMI nulos eliminados
binary_df = binary_df.merge(static_df, on='REF', how='inner')

In [179]:
binary_df

,REF,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> R=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1",...,<0> R=12 ALSFRS-R=>=36 -1 <3> R=12 ALSFRS-R=>=36 -1,<0> R=12 ALSFRS-R=>=36 -1 <4> R=12 -1,Gender,Age at onset (years),Revised El Escorial Criteria,Onset,Diagnostic delay (months),BMI at 1st visit,MND familiar history,C9orf72
0,2,1,1,1,0,0,1,1,1,1,...,1,1,Male,54.92,Probable,Limbs,5.22,17.90,No,unknown
1,8,1,0,0,0,0,0,0,0,0,...,1,1,Male,84.80,PMA,Limbs,8.90,29.38,No,unknown
2,9,1,1,0,0,0,1,0,0,1,...,1,1,Male,72.17,Possible,Limbs,27.93,26.61,No,No
3,10,1,1,0,0,0,0,0,0,0,...,1,1,Male,65.76,Probable,Bulbar,1.18,25.96,No,unknown
4,14,0,0,0,0,0,0,0,0,0,...,0,0,Female,61.63,Definite,Limbs,26.02,21.08,No,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,1794,1,0,0,0,0,0,0,0,0,...,0,0,Male,67.60,Possible,Limbs,6.05,21.26,No,No
624,1795,0,0,0,0,0,0,0,0,0,...,0,0,Female,37.22,Probable,Limbs,3.02,24.92,No,No
625,1814,1,0,0,0,0,0,0,0,0,...,1,1,Male,70.06,Prob. Lab Sup,Limbs,8.05,26.88,No,No
626,1818,0,0,0,0,0,0,0,0,0,...,0,0,Male,67.80,Probable,Bulbar,6.01,25.80,No,No


In [180]:
#one hot encode the static features with 0 and 1
binary_df = pd.get_dummies(binary_df, columns=['Gender', 'Revised El Escorial Criteria', 'Onset', 'MND familiar history', 'C9orf72'], dtype=int)

In [181]:
binary_df.head(3)

,REF,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> R=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1",...,Onset_Limbs,Onset_Respiratory,Onset_axial,Onset_diffuse,MND familiar history_No,MND familiar history_Yes,MND familiar history_unknown,C9orf72_No,C9orf72_Yes,C9orf72_unknown
0,2,1,1,1,0,0,1,1,1,1,...,1,0,0,0,1,0,0,0,0,1
1,8,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
2,9,1,1,0,0,0,1,0,0,1,...,1,0,0,0,1,0,0,1,0,0


In [182]:
#assign the evolution features to the binary_df
binary_df = binary_df.merge(evolution_df, on='REF', how='left')

In [183]:
binary_df.head(3)

,REF,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> R=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1",...,Onset_Respiratory,Onset_axial,Onset_diffuse,MND familiar history_No,MND familiar history_Yes,MND familiar history_unknown,C9orf72_No,C9orf72_Yes,C9orf72_unknown,Evolution
0,2,1,1,1,0,0,1,1,1,1,...,0,0,0,1,0,0,0,0,1,N
1,8,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,Y
2,9,1,1,0,0,0,1,0,0,1,...,0,0,0,1,0,0,1,0,0,Y


In [184]:
binary_df['Evolution'] = (binary_df['Evolution'] == 'Y').astype(int)

In [185]:
binary_df.head(3)

,REF,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> R=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1",...,Onset_Respiratory,Onset_axial,Onset_diffuse,MND familiar history_No,MND familiar history_Yes,MND familiar history_unknown,C9orf72_No,C9orf72_Yes,C9orf72_unknown,Evolution
0,2,1,1,1,0,0,1,1,1,1,...,0,0,0,1,0,0,0,0,1,0
1,8,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
2,9,1,1,0,0,0,1,0,0,1,...,0,0,0,1,0,0,1,0,0,1


Write text in cells like this ...


In [186]:
#count the number of patients with evolution 0 and 1
evolution_counts = binary_df['Evolution'].value_counts()
print(f"\nEvolution counts:\n{evolution_counts}")



Evolution counts:
Evolution
0    321
1    307
Name: count, dtype: int64


### 2.2. Create the Training Set

See Section **3.4 Training Set Creation and Model Learning** of the paper Martins et al (2021). 

Note that in this project the original static features are used thus you only need to compute the distance matrix for the sequencial patterns (which are now features), the static features are used as they are.

**Perform the experiments only for binary matrices**.

In [187]:
X = binary_df.drop(columns='Evolution')
y = binary_df['Evolution']

# one‐hot encode all categorical columns
#X_enc = pd.get_dummies(X, drop_first=True, dtype=int)



In [188]:
X.head(2)

,REF,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRS-R=>=36 -1 <4> R=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 ALSFRS-R=>=36 -1,<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSb=12 R=12 -1,"<0> ALSFRS-R=>=36 -1 <1> ALSFRS-R=>=36 -1 <2> ALSFRS-R=>=36 -1 <3> ALSFRSsT=[4,8[ -1",...,Onset_Limbs,Onset_Respiratory,Onset_axial,Onset_diffuse,MND familiar history_No,MND familiar history_Yes,MND familiar history_unknown,C9orf72_No,C9orf72_Yes,C9orf72_unknown
0,2,1,1,1,0,0,1,1,1,1,...,1,0,0,0,1,0,0,0,0,1
1,8,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1


In [189]:
#print the distribution of the target variable
# print(full_df['Evolution'].value_counts(normalize=True))
# print(y.value_counts(normalize=True))
print(f"Number of refs: {len(binary_df)}")

Number of refs: 628


Write text in cells like this ...


#### 2.1.3. Learn the model

See section **4.1 Model Evaluation** of the paper Martins et al (2021).

Use only a `Random-Forest`, default parameters and present the results for **5-fold cross-validation** (mean+-std). 

Note that the problem is difficult so don´t expect high performance.

In [190]:
# define RF and stratified 5‐fold CV
rf = RandomForestClassifier(random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# run CV on encoded data
#scores = cross_val_score(rf, X_enc, y, cv=cv, scoring='accuracy')
scores = cross_val_score(rf, X, y, cv=cv, scoring='accuracy')
print(f"5‐fold CV accuracy: {scores.mean():.3f} ± {scores.std():.3f}")

5‐fold CV accuracy: 0.557 ± 0.058


Write text in cells like this 

## Experiments

### Minimum Support 20%

In [ ]:
os.system("java -jar spmf.jar run Fournier08-Closed+time sequences.txt output_20.txt 20% 0 5 0 5")

In [ ]:
input_file = 'output.txt'
output_file = 'filtered_output.txt'

with open(input_file, 'r', encoding='utf-8') as fin, \
     open(output_file, 'w', encoding='utf-8') as fout:
    for line in fin:
        # collect all time‐point tags like <0>, <1>
        tps = re.findall(r'<\s*(\d+)\s*>', line)
        # if exactly one time‐point and it's "0", skip this line
        if len(tps) == 1 and tps[0] == '0':
            continue
        fout.write(line)

print(f"Filtered file written to {output_file}")

In [ ]:
import re

def is_subsequence(pat, seq):
    i = 0
    for pset in pat:
        while i < len(seq) and not pset.issubset(seq[i]):
            i += 1
        if i == len(seq): return False
        i += 1
    return True

def parse_sequence(raw):
    out = []
    # find every "(t, …)" chunk
    for content in re.findall(r'\(\s*\d+,\s*(.*?)\s*\)', raw):
        toks = content.split()
        s = set()
        # every three tokens are feat, '=', val
        for i in range(0, len(toks), 3):
            feat, _, val = toks[i : i+3]
            s.add(f"{feat}={val}")
        out.append(s)
    return out

# then your matching block stays the same:
pattern_records = []
with open('filtered_output.txt','r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line.startswith('<'): continue

        pat_str, sup_str = line.split('#SUP:')
        enc_pattern = pat_str.strip()
        support     = int(sup_str.strip())

        # build decoded pattern
        sets = re.findall(r'<(\d+)>\s+(.*?)\s+-1', enc_pattern)
        decoded = " ".join(
            f"({t}, {' '.join(f'{f} = {v}' for f,v in (x.split('=',1),))} )"
            for t,items in sets
            for x in [" ".join(items.split())]  # no-op to line up formats
        )
        # parse the encoded sets for subsequence
        pat_sets = [ set(s.split()) for _,s in sets ]

        hits = [
            seq_row.REF
            for _, seq_row in sequences.iterrows()
            if is_subsequence(pat_sets, parse_sequence(seq_row.Sequence_raw))
        ]

        pattern_records.append({
            'pattern':         enc_pattern,
            'decoded_pattern': decoded,
            'support':         support,
            'REF':            hits
        })

df_matches = pd.DataFrame(pattern_records)

df_matches = df_matches.sort_values(
    by='support',
    ascending=False
).reset_index(drop=True)

total_sequences = len(sequences)

df_matches['support_pct'] = (df_matches['support'] / total_sequences * 100).round(2)